In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing
import xgboost as xgb

In [2]:
mac_path = "/Users/jingweizhang/Dropbox/DataScience/Kaggle/CaterpillarTubePrice/competitions/caterpillar-tube-pricing/competition_data"
train = pd.read_csv(mac_path + '/train_set.csv', parse_dates=[2,])
test = pd.read_csv(mac_path + '/test_set.csv', parse_dates=[3,])
tube_data = pd.read_csv(mac_path + '/tube.csv')
bill_of_materials_data = pd.read_csv(mac_path + '/bill_of_materials.csv')
specs_data = pd.read_csv(mac_path + '/specs.csv')

In [3]:
print("train columns")
print(train.columns)
print("test columns")
print(test.columns)
print("tube.csv df columns")
print(tube_data.columns)
print("bill_of_materials.csv df columns")
print(bill_of_materials_data.columns)
print("specs.csv df columns")
print(specs_data.columns)

print(specs_data[2:3])

train columns
Index(['tube_assembly_id', 'supplier', 'quote_date', 'annual_usage',
       'min_order_quantity', 'bracket_pricing', 'quantity', 'cost'],
      dtype='object')
test columns
Index(['id', 'tube_assembly_id', 'supplier', 'quote_date', 'annual_usage',
       'min_order_quantity', 'bracket_pricing', 'quantity'],
      dtype='object')
tube.csv df columns
Index(['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x',
       'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other'],
      dtype='object')
bill_of_materials.csv df columns
Index(['tube_assembly_id', 'component_id_1', 'quantity_1', 'component_id_2',
       'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4',
       'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6',
       'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8',
       'quantity_8'],
      dtype='object')
specs.csv df columns
Index(['

In [4]:
train = pd.merge(train, tube_data, on = 'tube_assembly_id')
train = pd.merge(train, bill_of_materials_data, on ='tube_assembly_id')
test = pd.merge(test, tube_data, on ='tube_assembly_id')
test = pd.merge(test, bill_of_materials_data, on ='tube_assembly_id')

print("new train columns")
# print(train.columns)
# print(train[1:10])
print(train.columns.to_series().groupby(train.dtypes).groups)


new train columns
{dtype('<M8[ns]'): Index(['quote_date'], dtype='object'), dtype('int64'): Index(['annual_usage', 'min_order_quantity', 'quantity', 'num_bends',
       'num_boss', 'num_bracket', 'other'],
      dtype='object'), dtype('float64'): Index(['cost', 'diameter', 'wall', 'length', 'bend_radius', 'quantity_1',
       'quantity_2', 'quantity_3', 'quantity_4', 'quantity_5', 'quantity_6',
       'quantity_7', 'quantity_8'],
      dtype='object'), dtype('O'): Index(['tube_assembly_id', 'supplier', 'bracket_pricing', 'material_id',
       'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x',
       'component_id_1', 'component_id_2', 'component_id_3', 'component_id_4',
       'component_id_5', 'component_id_6', 'component_id_7', 'component_id_8'],
      dtype='object')}


In [5]:
# create some new features
train['year'] = train.quote_date.dt.year
train['month'] = train.quote_date.dt.month

test['year'] = test.quote_date.dt.year
test['month'] = test.quote_date.dt.month

# drop useless columns and create labels
idx = test.id.values.astype(int)
test = test.drop(['id', 'tube_assembly_id', 'quote_date'], axis = 1)
labels = train.cost.values

train = train.drop(['quote_date', 'cost', 'tube_assembly_id'], axis = 1)
print(train.columns)

Index(['supplier', 'annual_usage', 'min_order_quantity', 'bracket_pricing',
       'quantity', 'material_id', 'diameter', 'wall', 'length', 'num_bends',
       'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a',
       'end_x', 'num_boss', 'num_bracket', 'other', 'component_id_1',
       'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3',
       'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5',
       'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7',
       'quantity_7', 'component_id_8', 'quantity_8', 'year', 'month'],
      dtype='object')


In [6]:
train['material_id'].replace(np.nan,' ', regex=True, inplace= True)
test['material_id'].replace(np.nan,' ', regex=True, inplace= True)

In [7]:
for i in range(1, 9):
    column_label = 'component_id_' + str(i)
    train[column_label].replace(np.nan,' ', regex=True, inplace= True)
    test[column_label].replace(np.nan,' ', regex=True, inplace= True)
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

print("train columns")
print(train.columns)

# convert data to numpy array
train = np.array(train)
test = np.array(test)


train columns
Index(['supplier', 'annual_usage', 'min_order_quantity', 'bracket_pricing',
       'quantity', 'material_id', 'diameter', 'wall', 'length', 'num_bends',
       'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a',
       'end_x', 'num_boss', 'num_bracket', 'other', 'component_id_1',
       'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3',
       'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5',
       'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7',
       'quantity_7', 'component_id_8', 'quantity_8', 'year', 'month'],
      dtype='object')


In [8]:
# label encode the categorical variables
for i in range(train.shape[1]):
    if i in [0,3,5,11,12,13,14,15,16,20,22,24,26,28,30,32,34]:
        print(i,list(train[1:5,i]) + list(test[1:5,i]))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[:,i]) + list(test[:,i]))
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])
        
# object array to float
train = train.astype(float)
test = test.astype(float)

label_log = np.log1p(labels)

# fit a random forest model

params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["min_child_weight"] = 6
params["subsample"] = 0.87
params["colsample_bytree"] = 0.50
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 7

0 ['S-0066', 'S-0066', 'S-0066', 'S-0066', 'S-0066', 'S-0066', 'S-0066', 'S-0066']
3 ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']
5 ['SP-0019', 'SP-0019', 'SP-0019', 'SP-0019', 'SP-0035', 'SP-0035', 'SP-0035', 'SP-0035']
11 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
12 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
13 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
14 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
15 ['EF-008', 'EF-008', 'EF-008', 'EF-008', 'EF-003', 'EF-003', 'EF-003', 'EF-003']
16 ['EF-008', 'EF-008', 'EF-008', 'EF-008', 'EF-003', 'EF-003', 'EF-003', 'EF-003']
20 ['C-1312', 'C-1312', 'C-1312', 'C-1312', 'C-1622', 'C-1622', 'C-1622', 'C-1622']
22 [' ', ' ', ' ', ' ', 'C-1629', 'C-1629', 'C-1629', 'C-1629']
24 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
26 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
28 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
30 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
32 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
34 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
